# SITCOMTN-083 - Force Actuator Following Error Lag Analysis

As described in [SITCOM-1167], we are seeing delays in the Force Actuators applied forces when comparing them with the demanded forces.  
Now, we want to quantify those delays and have statistics on all the actuators.  
  
The [M1M3 Actuator Forces dashboard in USDF] contains the forces we want to use for this analysis.  
The url for this dashboard is created using the [m1m3_bump_test_times.py] Python script.  

This notebooks starts with mimicking [m1m3_bump_test_times.py] and bump tests analysis.  
Later, we will want to do similar analysis on slews (new ticket).  
We still need to think on how can we visualize this data (maybe new ticket).

[SITCOM-1167]: https://jira.lsstcorp.org/browse/SITCOM-1167
[M1M3 Actuator Forces dashboard in USDF]: https://usdf-rsp.slac.stanford.edu/chronograf/sources/1/dashboards/61?refresh=Paused&tempVars%5Bz_index%5D=112&tempVars%5By_index%5D=0&tempVars%5Bx_index%5D=112&tempVars%5Bs_index%5D=112&lower=now%28%29%20-%205m
[m1m3_bump_test_times.py]: https://github.com/lsst-ts/ts_criopy/blob/develop/python/lsst/ts/criopy/m1m3_bump_tests_times.py


## Notebook Preparations

Let's have here all the imports and global variables we will need during the notebook execution.  

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.time import Time
from pathlib import Path

from lsst.sitcom.vandv.logger import create_logger
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.summit.utils.tmaUtils import TMAEvent, TMAEventMaker
from lsst.ts.criopy.m1m3 import BumpTestTimes, ForceActuatorForces

from datetime import timezone
from pathlib import Path

In [ ]:
# Initial time to find out executions of bump tests
start_time = "2024-01-01T00:00:00"

# End time to find out executions of bump tests
end_time = "2024-01-10T00:00:00"

# List of force-actuators. If empty, look for them all
actuators = []

In [ ]:
# Create EFD Client
client = makeEfdClient()

# Initialize class to search for bump tests
bump_test_times = BumpTestTimes(client)